## Install

In [ ]:
pip install pandas

In [2]:
pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

2024-11-04 15:13:34.573238: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
pip install --upgrade accelerate

## Take in sample dataframe

In [4]:
bad_df = pd.read_csv("data/sample_bad.csv")
bad_df['label'] = 0
bad_df = bad_df[["link", "text", "label"]]
bad_df

,link,text,label
0,https://doi.org/10.1021%2Fjacs.5b04930,"HTTP Error, No Access",0
1,https://doi.org/10.1038%2Fnature14133,Compositional engineering of perovskite materi...,0
2,https://doi.org/10.1557%2Fmrc.2015.26,Identifying defect-tolerant semiconductors wit...,0
3,https://doi.org/10.1016%2F0927-0256%2896%29000...,Redirecting,0
4,https://doi.org/10.1021%2Facs.jpclett.6b02684,"HTTP Error, No Access",0
5,https://doi.org/10.1002%2Faenm.201601079,"HTTP Error, No Access",0
6,https://doi.org/10.1021%2Facsenergylett.6b00002,"HTTP Error, No Access",0
7,https://doi.org/10.1063%2F1.2903699,Effect of strain compensation on quantum dot e...,0
8,https://doi.org/10.1016%2Fj.solmat.2013.05.052,Redirecting,0
9,https://doi.org/10.4028%2Fwww.scientific.net%2...,Stress Gradient Analysis by Noncomplanar x-Ray...,0


In [5]:
pass_bad_df = bad_df[(bad_df['text'] != "HTTP Error, No Access") & (bad_df['text'] != "Redirecting")]
pass_bad_df

,link,text,label
1,https://doi.org/10.1038%2Fnature14133,Compositional engineering of perovskite materi...,0
2,https://doi.org/10.1557%2Fmrc.2015.26,Identifying defect-tolerant semiconductors wit...,0
7,https://doi.org/10.1063%2F1.2903699,Effect of strain compensation on quantum dot e...,0
9,https://doi.org/10.4028%2Fwww.scientific.net%2...,Stress Gradient Analysis by Noncomplanar x-Ray...,0
10,https://doi.org/10.1038%2Fnmat4014,Solvent engineering for high-performance inorg...,0
11,https://doi.org/10.1038%2Fnenergy.2016.93,Facet-dependent photovoltaic efficiency variat...,0
13,https://doi.org/10.1038%2Fs41467-017-00516-5,Strain-engineered growth of two-dimensional ma...,0
14,https://doi.org/10.1038%2Fncomms16045,Meniscus-assisted solution printing of large-g...,0
15,https://doi.org/10.1038%2Fnatrevmats.2018.17,Scalable fabrication of perovskite solar cells...,0
16,https://doi.org/10.1039%2FC7TA01441D,The intrinsic properties of FA(1−x)MAxPbI3 per...,0


In [6]:
good_df = pd.read_csv("data/sample_good.csv")
good_df['label'] = 1
good_df

,link,text,label
0,https://www.nature.com/articles/s41560-018-0200-6,A fluorene-terminated hole-transporting materi...,1
1,https://doi.org/10.1038%2Fs41586-021-03406-5,Pseudo-halide anion engineering for α-FAPbI3 p...,1
2,https://www.science.org/doi/10.1126/science.ab...,"HTTP Error, No Access",1
3,https://doi.org/10.1038%2Fs41560-023-01204-z,Expanding the low-dimensional interface engine...,1
4,https://pubs.acs.org/doi/10.1021/jacs.5b11740,"HTTP Error, No Access",1
5,https://doi.org/10.1002%2Faenm.201803699,"HTTP Error, No Access",1
6,https://doi.org/10.1126%2Fscience.adg3755,"HTTP Error, No Access",1
7,https://doi.org/10.1038%2Fs41586-021-03285-w,Efficient perovskite solar cells via improved ...,1
8,https://doi.org/10.1038%2Fs41566-022-00985-1,Quantum-size-tuned heterostructures enable eff...,1
9,https://doi.org/10.1039%2FC9TA06058H,Thermal degradation of formamidinium based lea...,1


In [7]:
pass_good_df = good_df[good_df['text'] != "HTTP Error, No Access"]
pass_good_df

,link,text,label
0,https://www.nature.com/articles/s41560-018-0200-6,A fluorene-terminated hole-transporting materi...,1
1,https://doi.org/10.1038%2Fs41586-021-03406-5,Pseudo-halide anion engineering for α-FAPbI3 p...,1
3,https://doi.org/10.1038%2Fs41560-023-01204-z,Expanding the low-dimensional interface engine...,1
7,https://doi.org/10.1038%2Fs41586-021-03285-w,Efficient perovskite solar cells via improved ...,1
8,https://doi.org/10.1038%2Fs41566-022-00985-1,Quantum-size-tuned heterostructures enable eff...,1
9,https://doi.org/10.1039%2FC9TA06058H,Thermal degradation of formamidinium based lea...,1
10,https://doi.org/10.1039%2FC8TC04871A,Amine additive reactions induced by the soft L...,1
13,https://pubs.rsc.org/en/content/articlehtml/20...,Stable perovskite solar cells using tin acetyl...,1
14,https://doi.org/10.1038%2Fs41560-023-01295-8,2D/3D heterojunction engineering at the buried...,1


# Combine these two dataframe

In [8]:
# merged_df = pass_good_df.append(pass_bad_df, ignore_index=True)[['text', 'label']]
# merged_df

merged_df = pd.concat([pass_good_df, pass_bad_df], ignore_index=True)[['text', 'label']]
merged_df

,text,label
0,A fluorene-terminated hole-transporting materi...,1
1,Pseudo-halide anion engineering for α-FAPbI3 p...,1
2,Expanding the low-dimensional interface engine...,1
3,Efficient perovskite solar cells via improved ...,1
4,Quantum-size-tuned heterostructures enable eff...,1
5,Thermal degradation of formamidinium based lea...,1
6,Amine additive reactions induced by the soft L...,1
7,Stable perovskite solar cells using tin acetyl...,1
8,2D/3D heterojunction engineering at the buried...,1
9,Compositional engineering of perovskite materi...,0


## Test 1 (Commented out but first test)

In [11]:
#first test run.
# import pandas as pd
# from datasets import Dataset, DatasetDict
# from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
# from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# data = pd.read_csv("data/merged_label.csv")

# data = data[['text', 'label']]

# dataset = Dataset.from_pandas(data)

# train_test_split = dataset.train_test_split(test_size=0.2)

# tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")
# model = AutoModelForSequenceClassification.from_pretrained("allenai/scibert_scivocab_uncased", num_labels=2)

# def tokenize_function(examples):
#     return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=256)

# tokenized_dataset = train_test_split.map(tokenize_function, batched=True)

# training_args = TrainingArguments(
#     output_dir='./results',
#     eval_strategy="epoch",
#     learning_rate=3e-5,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     num_train_epochs=3,
#     weight_decay=0.01,
#     logging_steps=500
# )

# def compute_metrics(p):
#     preds = p.predictions.argmax(axis=1)
#     precision, recall, f1, _ = precision_recall_fscore_support(p.label_ids, preds, average='binary')
#     acc = accuracy_score(p.label_ids, preds)
#     return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_dataset['train'],
#     eval_dataset=tokenized_dataset['test'],
#     compute_metrics=compute_metrics,
# )

# trainer.train()

# results = trainer.evaluate()
# print(results)

# predictions = trainer.predict(tokenized_dataset['test'])
# pred_labels = predictions.predictions.argmax(axis=1)
# true_labels = predictions.label_ids

# output_df = pd.DataFrame({'Text': tokenized_dataset['test']['text'], 'True Label': true_labels, 'Predicted Label': pred_labels})
# print(output_df)

## SciBERT Test 2 + output to csv

In [18]:
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

data = pd.read_csv("data/merged_label.csv")
data = data[['text', 'label']]
dataset = Dataset.from_pandas(data)

train_test_split = dataset.train_test_split(test_size=0.2)

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")
model = AutoModelForSequenceClassification.from_pretrained("allenai/scibert_scivocab_uncased", num_labels=2)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=256)

# Tokenize the dataset
tokenized_dataset = train_test_split.map(tokenize_function, batched=True)

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=500
)

# Compute metrics
def compute_metrics(p):
    preds = p.predictions.argmax(axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(p.label_ids, preds, average='binary')
    acc = accuracy_score(p.label_ids, preds)
    
    # Calculate Balanced Error Rate (BER)
    FP = ((preds == 1) & (p.label_ids == 0)).sum()
    FN = ((preds == 0) & (p.label_ids == 1)).sum()
    P = (p.label_ids == 1).sum()  # Positive samples
    N = (p.label_ids == 0).sum()  # Negative samples
    
    # Avoid division by zero
    if P + N == 0:
        ber = None
    else:
        ber = (FP / P + FN / N) / 2 if P > 0 and N > 0 else None
    
    return {"accuracy": acc, "recall": recall, "ber": ber}

# Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

# Evaluate the model
results = trainer.evaluate()
print(results)

# Make predictions
predictions = trainer.predict(tokenized_dataset['test'])
pred_labels = predictions.predictions.argmax(axis=1)
true_labels = predictions.label_ids

# Prepare output DataFrame
output_df = pd.DataFrame({
    'model_name': ['scibert'],  # Change this to your model name
    'BER': [results['eval_ber']],
    'recall': [results['eval_recall']],
    'accuracy': [results['eval_accuracy']]
})

# Save to CSV
output_df.to_csv('model_metrics.csv', index=False)

print(output_df)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at allenai/scibert_scivocab_uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/105 [00:00<?, ? examples/s]

Map:   0%|          | 0/27 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,Recall,Ber
1,No log,0.617458,0.740741,0.846154,0.263736
2,No log,0.564529,0.703704,0.692308,0.296703
3,No log,0.606332,0.703704,0.769231,0.299451


{'eval_loss': 0.606332004070282, 'eval_accuracy': 0.7037037037037037, 'eval_recall': 0.7692307692307693, 'eval_ber': 0.29945054945054944, 'eval_runtime': 12.6439, 'eval_samples_per_second': 2.135, 'eval_steps_per_second': 0.158, 'epoch': 3.0}
  model_name       BER    recall  accuracy
0    scibert  0.299451  0.769231  0.703704
